In [1]:
from __future__ import print_function

import tensorflow as tf
import numpy as np
from tensorflow.contrib import rnn

tf.set_random_seed(777)  # reproducibility

In [2]:
sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {w: i for i, w in enumerate(char_set)}

In [4]:
print(char_set)
print(char_dic)

['s', 'k', 'n', 'i', 'h', 't', 'w', 'd', 'u', ' ', 'b', 'p', 'l', 'm', '.', 'f', 'e', 'r', 'a', ',', 'o', "'", 'g', 'y', 'c']
{'d': 7, 's': 0, 'p': 11, 'l': 12, 'm': 13, '.': 14, 'f': 15, 'e': 16, 'k': 1, 'o': 20, 'r': 17, 'n': 2, 'i': 3, 'h': 4, 't': 5, 'a': 18, ',': 19, 'w': 6, "'": 21, 'u': 8, 'g': 22, 'y': 23, ' ': 9, 'b': 10, 'c': 24}


In [5]:
data_dim = len(char_set)
hidden_size = len(char_set)
num_classes = len(char_set)
sequence_length = 10  # Any arbitrary number
learning_rate = 0.1

dataX = []
dataY = []
for i in range(0, len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i + 1: i + sequence_length + 1]
    print(i, x_str, '->', y_str)

    x = [char_dic[c] for c in x_str]  # x str to index
    y = [char_dic[c] for c in y_str]  # y str to index

    dataX.append(x)
    dataY.append(y)

batch_size = len(dataX)

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [6]:
X = tf.placeholder(tf.int32, [None, sequence_length])
Y = tf.placeholder(tf.int32, [None, sequence_length])

# One-hot encoding
X_one_hot = tf.one_hot(X, num_classes)
print(X_one_hot)  # check out the shape


# Make a lstm cell with hidden_size (each unit output vector size)
def lstm_cell():
    cell = rnn.BasicLSTMCell(hidden_size, state_is_tuple=True)
    return cell

multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(2)], state_is_tuple=True)

# outputs: unfolding size x hidden size, state = hidden size
outputs, _states = tf.nn.dynamic_rnn(multi_cells, X_one_hot, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(X_for_fc, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

# All weights are 1 (equal weights)
weights = tf.ones([batch_size, sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(
    logits=outputs, targets=Y, weights=weights)
mean_loss = tf.reduce_mean(sequence_loss)
train_op = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(mean_loss)

Tensor("one_hot:0", shape=(?, 10, 25), dtype=float32)


In [7]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(500):
    _, l, results = sess.run(
        [train_op, mean_loss, outputs], feed_dict={X: dataX, Y: dataY})
    for j, result in enumerate(results):
        index = np.argmax(result, axis=1)
        print(i, j, ''.join([char_set[t] for t in index]), l)

# Let's print the last char of each result to check it works
results = sess.run(outputs, feed_dict={X: dataX})
for j, result in enumerate(results):
    index = np.argmax(result, axis=1)
    if j is 0:  # print all for the first result to make a sentence
        print(''.join([char_set[t] for t in index]), end='')
    else:
        print(char_set[index[-1]], end='')

0 0 nnsssiiiii 3.21675
0 1 gggg,,,,,, 3.21675
0 2 ggl,li,i,, 3.21675
0 3 ,,iiiiii,, 3.21675
0 4 iiiiiii,,, 3.21675
0 5 iiiii,,,,, 3.21675
0 6 g,,,,,,,,, 3.21675
0 7 ari,,,,,,, 3.21675
0 8 ,,,,,,,,,, 3.21675
0 9 i,,,,,,,ii 3.21675
0 10 b,,,,,,,,, 3.21675
0 11 g,,lllii,, 3.21675
0 12 bb,,iiii,, 3.21675
0 13 iiiiiii',, 3.21675
0 14 ggiiii,,,, 3.21675
0 15 iiiiii,,,, 3.21675
0 16 iin'',,,,, 3.21675
0 17 n,,,,,,,,, 3.21675
0 18 ,,,,,,,,,, 3.21675
0 19 b,,,,,,,,, 3.21675
0 20 gggg,,,,,, 3.21675
0 21 ,g,,n,nnt' 3.21675
0 22 glllln'''' 3.21675
0 23 yynnnn'''' 3.21675
0 24 nnnnn''',, 3.21675
0 25 nssst,,,,, 3.21675
0 26 sss,,,,,,, 3.21675
0 27 ngg,,,,,,, 3.21675
0 28 gg,,,,,,,, 3.21675
0 29 bb,,,,,,,, 3.21675
0 30 iii,,,,,,, 3.21675
0 31 i,,,,,,,,, 3.21675
0 32 ,,,,,,,,,, 3.21675
0 33 b,,,,,,,,, 3.21675
0 34 gg,,,,,''' 3.21675
0 35 b,a,,''''' 3.21675
0 36    e'''''' 3.21675
0 37 i''''''''' 3.21675
0 38 'lllll,,,, 3.21675
0 39 ggggg,,,,, 3.21675
0 40 ioss,,,,,, 3.21675
0 41 sgs,,,,,,, 3.21675
0 

10 0            2.61122
10 1            2.61122
10 2            2.61122
10 3            2.61122
10 4            2.61122
10 5            2.61122
10 6            2.61122
10 7            2.61122
10 8            2.61122
10 9            2.61122
10 10            2.61122
10 11            2.61122
10 12            2.61122
10 13            2.61122
10 14            2.61122
10 15            2.61122
10 16         ee 2.61122
10 17        eee 2.61122
10 18         ee 2.61122
10 19        ee  2.61122
10 20       ee   2.61122
10 21      ee    2.61122
10 22     ee     2.61122
10 23    ee      2.61122
10 24 e e        2.61122
10 25            2.61122
10 26            2.61122
10 27            2.61122
10 28            2.61122
10 29            2.61122
10 30            2.61122
10 31          e 2.61122
10 32        eee 2.61122
10 33       eeee 2.61122
10 34       eeee 2.61122
10 35      eeeee 2.61122
10 36  e eeeeeee 2.61122
10 37        e e 2.61122
10 38       e ee 2.61122
10 39        eee 2.61122
10 40     

20 0   tot aond 1.36199
20 1  tot wond  1.36199
20 2 tot nond t 1.36199
20 3  t nond to 1.36199
20 4 n nond tor 1.36199
20 5    ndothrd 1.36199
20 6 tond tordu 1.36199
20 7 ond tordui 1.36199
20 8 sd torduil 1.36199
20 9 d torduild 1.36199
20 10 orerbuild  1.36199
20 11 torduild a 1.36199
20 12 o duild a, 1.36199
20 13 nduild a,s 1.36199
20 14 tuild a,si 1.36199
20 15 uild a,sii 1.36199
20 16  pdoa,ssip 1.36199
20 17  sorssiip, 1.36199
20 18 e nssinp,  1.36199
20 19  ans np, l 1.36199
20 20 tngnnp, do 1.36199
20 21 ssiip, don 1.36199
20 22 tiipn don' 1.36199
20 23 i p, ton't 1.36199
20 24 etn lotdt  1.36199
20 25  , ton't t 1.36199
20 26 e lon't to 1.36199
20 27  'o''t aoa 1.36199
20 28 ton't toam 1.36199
20 29  n't aoam  1.36199
20 30 nd  ooam u 1.36199
20 31 dt toam up 1.36199
20 32 t aoam up  1.36199
20 33 oroam up p 1.36199
20 34 toam up pe 1.36199
20 35   m up pee 1.36199
20 36  m up peep 1.36199
20 37   u  peepo 1.36199
20 38  t  peepoe 1.36199
20 39 t  peepoe  1.36199
20 40   pe

30 0 g tot want 0.517982
30 1  tou want  0.517982
30 2 tou want t 0.517982
30 3  u want to 0.517982
30 4 n want to  0.517982
30 5 pwant to b 0.517982
30 6 tont to bu 0.517982
30 7 ont to bui 0.517982
30 8 nd to buil 0.517982
30 9 d to build 0.517982
30 10 hto build  0.517982
30 11 th luild a 0.517982
30 12 h luild a  0.517982
30 13 nluild a s 0.517982
30 14 tuild a sh 0.517982
30 15 uild a shi 0.517982
30 16 pld anship 0.517982
30 17 gd aaship, 0.517982
30 18 eodnship,  0.517982
30 19 oanship, d 0.517982
30 20 tnship, do 0.517982
30 21 nship, don 0.517982
30 22 tiip, don' 0.517982
30 23 iip, don't 0.517982
30 24 em, don't  0.517982
30 25 g, don't a 0.517982
30 26 e bon't ar 0.517982
30 27  don't aru 0.517982
30 28 ton't arum 0.517982
30 29 on't arum  0.517982
30 30 n't drum u 0.517982
30 31 dt drum up 0.517982
30 32 t arum up  0.517982
30 33 htoum up p 0.517982
30 34 toum up pe 0.517982
30 35 oum up peo 0.517982
30 36  m up peop 0.517982
30 37 p up peopl 0.517982
30 38  tp people 0.517

40 0 p you want 0.331499
40 1  tou want  0.331499
40 2 tou want t 0.331499
40 3 ou want to 0.331499
40 4   want to  0.331499
40 5 iwant to b 0.331499
40 6 tont to bu 0.331499
40 7 ont to bui 0.331499
40 8 nd to buil 0.331499
40 9 d to build 0.331499
40 10 hta build  0.331499
40 11 to build a 0.331499
40 12 h build a  0.331499
40 13  cuild a s 0.331499
40 14 tutld a sh 0.331499
40 15 utld a shi 0.331499
40 16 ild a ship 0.331499
40 17 pd a ship, 0.331499
40 18 eoasship,  0.331499
40 19  anship, d 0.331499
40 20 tnship, do 0.331499
40 21 nship, don 0.331499
40 22 thip, don' 0.331499
40 23  ip, don't 0.331499
40 24 ep, don't  0.331499
40 25 p, don't d 0.331499
40 26 , don't dr 0.331499
40 27  don't dru 0.331499
40 28 ton't drum 0.331499
40 29  n't drum  0.331499
40 30  't arum u 0.331499
40 31 dt arum up 0.331499
40 32 t drum up  0.331499
40 33 htrum up p 0.331499
40 34 toum up pe 0.331499
40 35  um up peo 0.331499
40 36  m up peop 0.331499
40 37 i up peopl 0.331499
40 38  tp people 0.331

...

461 124  ach them  0.22876
461 125 rch them t 0.22876
461 126 nh them to 0.22876
461 127 o them to  0.22876
461 128 ethem to l 0.22876
461 129 toem to lo 0.22876
461 130  er to lon 0.22876
461 131 er to long 0.22876
461 132 r to long  0.22876
461 133  to long f 0.22876
461 134 to bong fo 0.22876
461 135   bong for 0.22876
461 136 nbong for  0.22876
461 137 tong for t 0.22876
461 138 eng for th 0.22876
461 139 n' for the 0.22876
461 140 d for the  0.22876
461 141  for the e 0.22876
461 142 tor the en 0.22876
461 143 or the end 0.22876
461 144 nkthe endl 0.22876
461 145  toe endle 0.22876
461 146 toemendles 0.22876
461 147  erendless 0.22876
461 148 erendless  0.22876
461 149 rendless i 0.22876
461 150 tndless im 0.22876
461 151 rsless imm 0.22876
461 152 d ess imme 0.22876
461 153  ess immen 0.22876
461 154 ess immens 0.22876
461 155 rs immensi 0.22876
461 156 siimmensit 0.22876
461 157 simmensity 0.22876
461 158 tmmensity  0.22876
461 159 gmensity o 0.22876
461 160  ensity of 0.22876
4

469 0 t you want 0.228754
469 1  you want  0.228754
469 2 tou want t 0.228754
469 3 ou want to 0.228754
469 4   want to  0.228754
469 5  want to b 0.228754
469 6 tont to bu 0.228754
469 7 ont to bui 0.228754
469 8 nd to buil 0.228754
469 9 d to build 0.228754
469 10  ao build  0.228754
469 11 to build a 0.228754
469 12   build a  0.228754
469 13  build a s 0.228754
469 14 tutld a sh 0.228754
469 15 utld a shi 0.228754
469 16  ld a ship 0.228754
469 17 td a ship, 0.228754
469 18 e a ship,  0.228754
469 19  anship, d 0.228754
469 20 tnship, do 0.228754
469 21 nship, don 0.228754
469 22 thip, don' 0.228754
469 23  ip, don't 0.228754
469 24 ep, don't  0.228754
469 25 t, don't d 0.228754
469 26   don't dr 0.228754
469 27  bon't dru 0.228754
469 28 ton't arum 0.228754
469 29  n't arum  0.228754
469 30  't arum u 0.228754
469 31 dt arum up 0.228754
469 32 t arum up  0.228754
469 33  arum up p 0.228754
469 34 toum up pe 0.228754
469 35  um up peo 0.228754
469 36  m up peop 0.228754
469 37   up

479 0 t you want 0.229678
479 1  you want  0.229678
479 2 tou want t 0.229678
479 3  u want to 0.229678
479 4 n want to  0.229678
479 5  want to b 0.229678
479 6 tont to bu 0.229678
479 7 ont to bui 0.229678
479 8 nd to buil 0.229678
479 9 d to build 0.229678
479 10  ao build  0.229678
479 11 to build a 0.229678
479 12   build a  0.229678
479 13 nbuild a s 0.229678
479 14 tutld a sh 0.229678
479 15 utld a shi 0.229678
479 16  ld a ship 0.229678
479 17 td a ship, 0.229678
479 18 e a ship,  0.229678
479 19  anship, d 0.229678
479 20 tnship, do 0.229678
479 21 nship, don 0.229678
479 22 thip, don' 0.229678
479 23  ip, don't 0.229678
479 24 ep, don't  0.229678
479 25 t, don't d 0.229678
479 26   don't dr 0.229678
479 27  bon't dru 0.229678
479 28 ton't arum 0.229678
479 29  n't arum  0.229678
479 30 n't arum u 0.229678
479 31 dt arum up 0.229678
479 32 t arum up  0.229678
479 33  arum up p 0.229678
479 34 toum up pe 0.229678
479 35  um up peo 0.229678
479 36  m up peop 0.229678
479 37   up

489 0 t you want 0.228808
489 1  you want  0.228808
489 2 tou want t 0.228808
489 3 ou want to 0.228808
489 4   want to  0.228808
489 5  want to b 0.228808
489 6 tont to bu 0.228808
489 7 ont to bui 0.228808
489 8 nd to buil 0.228808
489 9 d to build 0.228808
489 10 hao build  0.228808
489 11 to cuild a 0.228808
489 12 h cuild a  0.228808
489 13  cuild a s 0.228808
489 14 tuild a sh 0.228808
489 15 utld a shi 0.228808
489 16  ld a ship 0.228808
489 17 td a ship, 0.228808
489 18 e a ship,  0.228808
489 19  anship, d 0.228808
489 20 tnship, do 0.228808
489 21 nship, don 0.228808
489 22 thip, don' 0.228808
489 23  ip, don't 0.228808
489 24 ep, don't  0.228808
489 25 t, don't d 0.228808
489 26   don't dr 0.228808
489 27  bon't dru 0.228808
489 28 ton't arum 0.228808
489 29  n't arum  0.228808
489 30  't arum u 0.228808
489 31 dt arum up 0.228808
489 32 t arum up  0.228808
489 33 harum up p 0.228808
489 34 toum up pe 0.228808
489 35  um up peo 0.228808
489 36  m up peop 0.228808
489 37   up

499 0 p you want 0.228757
499 1  you want  0.228757
499 2 tou want t 0.228757
499 3 ou want to 0.228757
499 4   want to  0.228757
499 5  want to b 0.228757
499 6 tont to bu 0.228757
499 7 ont to bui 0.228757
499 8 nd to buil 0.228757
499 9 d to build 0.228757
499 10 hao build  0.228757
499 11 to build a 0.228757
499 12 h build a  0.228757
499 13  cuild a s 0.228757
499 14 tuild a sh 0.228757
499 15 uild a shi 0.228757
499 16  ld a ship 0.228757
499 17 pd a ship, 0.228757
499 18 e a ship,  0.228757
499 19  anship, d 0.228757
499 20 tnship, do 0.228757
499 21 nship, don 0.228757
499 22 thip, don' 0.228757
499 23 sip, don't 0.228757
499 24 ep, don't  0.228757
499 25 p, don't d 0.228757
499 26 l don't dr 0.228757
499 27  bon't dru 0.228757
499 28 ton't arum 0.228757
499 29  n't drum  0.228757
499 30  't arum u 0.228757
499 31 dt arum up 0.228757
499 32 t drum up  0.228757
499 33 harum up p 0.228757
499 34 toum up pe 0.228757
499 35  um up peo 0.228757
499 36  m up peop 0.228757
499 37   up